In [1]:
import tensorflow as tf

In [ ]:
class Autoencoder(object):
    def __init__(self, inputs, hps):
        '''
        Projects the input into a hidden dimension, then down a bottleneck layer.
        Then projects the bottleneck layer back to the hidden dimension, before reconstructing the input.
        '''
        self.inputs = inputs
        self.input_dim = tf.shape(inputs)[1]
        self.hidden1_dim = hps.hidden1_dim
        self.bottleneck_dim = hps.bottleneck_dim
        
        # initializer for weights
        self.weight_init = tf.variance_scaling_initializer()
        # learning rate for optim
        self.lr = hps.lr
        
    def build_model(self):
        self.reconstruction()
        self.cost()
        self.train_op()
        
    def reconstruction(self):
        # project down to hidden bottleneck dimension
        x = tf.layers.dense(self.inputs, self.hidden1_dim, activation=tf.nn.tanh,
                            kernel_initializer=self.weight_init)
        x = tf.layers.dense(x, self.bottleneck_dim, activation=tf.nn.tanh,
                           kernel_initializer=self.weight_init)
        # project back to inputs
        x = tf.layers.dense(x, self.hidden1_dim, activation=tf.nn.tanh,
                           kernel_initializer=self.weight_init)
        # linear activation for inputs
        self.reconstruction(x, self.input_dim, kernel_initializer=self.weight_init)
        
    def cost(self):
        # reconstruction error as loss
        loss = (self.inputs - self.reconstruction) ** 2
        self.cost = tf.reduce_mean(loss)
        
    def train_op(self):
        optim = tf.train.MomentumOptimizer(learning_rate=self.lr, momentum=.9, use_nesterov=True)
        self.train_op = optim.minimize(self.cost)

In [ ]:
class DenoisingAutoencoder(object):
    def __init__(self, inputs, hps):
        '''
        Projects the input into a hidden dimension, then down a bottleneck layer.
        Then projects the bottleneck layer back to the hidden dimension, before reconstructing the input.
        '''
        self.inputs = inputs
        self.input_dim = tf.shape(inputs)[1]
        self.hidden1_dim = hps.hidden1_dim
        self.bottleneck_dim = hps.bottleneck_dim
        
        # initializer for weights
        self.weight_init = tf.variance_scaling_initializer()
        # learning rate for optim
        self.lr = hps.lr
        
    def build_model(self):
        self.reconstruction()
        self.cost()
        self.train_op()
        
    def reconstruction(self):
        # add noise to the inputs
        in_noise = tf.random_normal(tf.shape(self.inputs))
        noise_scale = .4
        x = self.inputs + noise_scale * in_noise
        # project down to hidden bottleneck dimension
        x = tf.layers.dense(x, self.hidden1_dim, activation=tf.nn.tanh,
                            kernel_initializer=self.weight_init)
        x = tf.layers.dense(x, self.bottleneck_dim, activation=tf.nn.tanh,
                           kernel_initializer=self.weight_init)
        # project back to inputs
        x = tf.layers.dense(x, self.hidden1_dim, activation=tf.nn.tanh,
                           kernel_initializer=self.weight_init)
        # linear activation for inputs
        self.reconstruction(x, self.input_dim, kernel_initializer=self.weight_init)
        
    def cost(self):
        # reconstruction error as loss
        loss = (self.inputs - self.reconstruction) ** 2
        self.cost = tf.reduce_mean(loss)
        
    def train_op(self):
        optim = tf.train.MomentumOptimizer(learning_rate=self.lr, momentum=.9, use_nesterov=True)
        self.train_op = optim.minimize(self.cost)

In [1]:
class VariationalAutoencoder(object):
    def __init__(self, inputs, hps):
        '''
        Projects the input into a hidden dimension, then down a bottleneck layer.
        Then projects the bottleneck layer back to the hidden dimension, before reconstructing the input.
        '''
        self.inputs = inputs
        self.input_dim = tf.shape(inputs)[1]
        self.hidden1_dim = hps.hidden1_dim
        self.latent_dim = 64 # hps.latent_dim
        
        # initializer for weights
        self.weight_init = tf.variance_scaling_initializer()
        # learning rate for optim
        self.lr = hps.lr
        
    def build_model(self):
        self.reconstruction()
        self.cost()
        self.train_op()
        
    def reconstruction(self):
        # project down to latent dimension
        x = tf.layers.dense(x, self.hidden1_dim, activation=tf.nn.relu,
                            kernel_initializer=self.weight_init)
        x = tf.layers.dense(x, self.hidden1_dim, activation=tf.nn.tanh,
                            kernel_initializer=self.weight_init)
        # learn parameters for distributions
        mu = tf.layers.dense(x, self.latent_dim,
                           kernel_initializer=self.weight_init)
        logstd = tf.layers.dense(x, self.latent_dim,
                           kernel_initializer=self.weight_init)
        # reparamterization trick
        aux_noise = in_noise = tf.random_normal(self.latent_dim)
        z = mu + aux_noise * tf.exp(0.5 * logstd)
        # project back to input dimensions
        x = tf.layers.dense(z, self.hidden1_dim, activation=tf.nn.tanh,
                           kernel_initializer=self.weight_init)
        x = tf.layers.dense(x, self.hidden1_dim, activation=tf.nn.relu,
                           kernel_initializer=self.weight_init)
        # linear activation for input reconstruction
        self.reconstruction = tf.layers.dense(x, self.input_dim, kernel_initializer=self.weight_init)
        # could also do sigmoid output for binary cross entropy
        #self.reconstruction = tf.layers.dense(x, self.input_dim, activation=tf.nn.sigmoid,
        #                                     kernel_initializer=self.weight_init)
        
    def cost(self):
        # reconstruction error as loss
        loss = (self.inputs - self.reconstruction) ** 2
        # binary cross entropy, assumes inputs are 0/1
        # eps = 1e-8
        #loss = self.inputs * tf.log(self.reconstruction + eps) + (1 - self.inputs) * tf.log(1 - self.reconstruction + eps)
        
        # KL divergence between learned parameters and uniform normal prior
        KL_term = -.5*tf.reduce_mean(1 + 2*self.logstd - tf.pow(self.mu,2) - tf.exp(2*self.logstd))
        # Combined cost + latent divergence to minimize
        self.cost = tf.reduce_mean(loss) + KL_term
        
    def train_op(self):
        optim = tf.train.MomentumOptimizer(learning_rate=self.lr, momentum=.9, use_nesterov=True)
        self.train_op = optim.minimize(self.cost)

In [ ]:
class ConvolutionalAutoencoder(object):
    def __init__(self, inputs, hps):
        '''
        Projects the input into a hidden dimension, then down a bottleneck layer.
        Then projects the bottleneck layer back to the hidden dimension, before reconstructing the input.
        '''
        self.inputs = inputs
        self.input_dim = tf.shape(inputs)[1]
        self.hidden1_dim = hps.hidden1_dim
        self.latent_dim = 64 # hps.latent_dim
        
        # initializer for weights
        self.weight_init = tf.variance_scaling_initializer()
        # learning rate for optim
        self.lr = hps.lr
        
    def build_model(self):
        self.reconstruction()
        self.cost()
        self.train_op()
        
    def reconstruction(self):
        # project down to latent dimension
        x = tf.layers.dense(x, self.hidden1_dim, activation=tf.nn.relu,
                            kernel_initializer=self.weight_init)
        x = tf.layers.dense(x, self.hidden1_dim, activation=tf.nn.tanh,
                            kernel_initializer=self.weight_init)
        # learn parameters for distributions
        mu = tf.layers.dense(x, self.latent_dim,
                           kernel_initializer=self.weight_init)
        logstd = tf.layers.dense(x, self.latent_dim,
                           kernel_initializer=self.weight_init)
        # reparamterization trick
        aux_noise = in_noise = tf.random_normal(self.latent_dim)
        z = mu + aux_noise * tf.exp(0.5 * logstd)
        # project back to input dimensions
        x = tf.layers.dense(z, self.hidden1_dim, activation=tf.nn.tanh,
                           kernel_initializer=self.weight_init)
        x = tf.layers.dense(x, self.hidden1_dim, activation=tf.nn.relu,
                           kernel_initializer=self.weight_init)
        # linear activation for input reconstruction
        self.reconstruction = tf.layers.dense(x, self.input_dim, kernel_initializer=self.weight_init)
        # could also do sigmoid output for binary cross entropy
        #self.reconstruction = tf.layers.dense(x, self.input_dim, activation=tf.nn.sigmoid,
        #                                     kernel_initializer=self.weight_init)
        
    def cost(self):
        # reconstruction error as loss
        loss = (self.inputs - self.reconstruction) ** 2
        # binary cross entropy, assumes inputs are 0/1
        # eps = 1e-8
        #loss = self.inputs * tf.log(self.reconstruction + eps) + (1 - self.inputs) * tf.log(1 - self.reconstruction + eps)
        
        # KL divergence between learned parameters and uniform normal prior
        KL_term = -.5*tf.reduce_mean(1 + 2*self.logstd - tf.pow(self.mu,2) - tf.exp(2*self.logstd))
        # Combined cost + latent divergence to minimize
        self.cost = tf.reduce_mean(loss) + KL_term
        
    def train_op(self):
        optim = tf.train.MomentumOptimizer(learning_rate=self.lr, momentum=.9, use_nesterov=True)
        self.train_op = optim.minimize(self.cost)